In [1]:
#Code to download all needed libraries 

from __future__ import division 
from pyomo.environ import * #library for modeling and optimisation
import argparse
from pyomo.opt import SolverStatus, TerminationCondition #provide status of solvers
import pandas as pd 
import numpy as np 
import pickle
import random
import sqlite3
import copy
import random
import csv

import warnings

import os
warnings.filterwarnings('ignore')

INTEREST = 0.03 #interest rate

path =   #specify the path to the working folder

In [2]:
#The main integer problem for running on stands and cells 

class optimization:
    def __init__(self):
        
        data_opt =  pd.read_parquet(path + "2025-03-18_cells_30.parquet") #adjust file name
        print("data loaded")
        all_data = data_opt
        all_data = all_data.set_index(['stand','schedule','period']) #reassigns the Df with new multi-level index
        
        all_data = all_data.fillna(0) #fills any missing values with 0
        all_data['period'] = all_data.index.get_level_values(2)
        
        self.data_opt=all_data #store the fetched or preprocessed data for use within the class
        self.combinations = 1
        self.all_data = self.data_opt
        self.Index_values = self.all_data.drop(['period'], axis=1).reset_index().set_index(['stand','schedule']).index.unique() #we drop year, because we dont make decision for each year
        self.area = self.all_data.loc[slice(None),1,1]['area']
        self.all_data = self.all_data.fillna(0)

        self.createModel()

    def createModel(self):
        # Declare sets - These used to recongnize the number of stands, regimes and number of periods in the analysis.
        # Define sets, variables, and constraints for the optimization problem
        
        self.model1 = ConcreteModel() #defining the model
        
        self.model1.stands = Set(initialize = list(set(self.all_data.index.get_level_values(0)))) #initialized with unique values from the first level of the index created earlier
        self.model1.year = Set(initialize = list(set(self.all_data.index.get_level_values(2)))) #initialized with unique values from the third level of the index created earlier
               
        self.model1.regimes = Set(initialize = list(set(self.all_data.index.get_level_values(1))))
        self.model1.scen_index = Set(initialize= [i for i in range(0,self.combinations)])
        self.model1.Index_values = self.Index_values

        #Indexes (stand, regime)-- excludes those combinations that have no regimes simulated -- because some regimes are not simulated for some stands

        def index_rule(model1):
            index = []
            for (s,r) in model1.Index_values: 
                index.append((s,r))
            return index
        self.model1.index1 = Set(dimen=2, initialize=index_rule)

        #Decision variable
        self.model1.X1 = Var(self.model1.index1, within=Boolean, initialize=1) 

        self.all_data['period'] = self.all_data.index.get_level_values(2)

        #Objective and constraint don't need to be adjusted here, leave as it is, used to create the optimization model in the code. 
        #objective function:
        def outcome_rule(model1):
            return sum((self.all_data.Harvested_V_S.loc[(s,r,k)]*self.all_data.area.loc[(s,r,k)]* self.model1.X1[(s,r)])/((1+INTEREST)**(2.5+self.all_data.period[(s,r,k)]))  for (s,r) in self.model1.index1 for k in self.model1.year)
        self.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)

        #Constraint to ensure only one regime prescription is chosen for each stand
        def regime_rule(model1, s):
            row_sum = sum(model1.X1[(s,r)] for r in [x[1] for x in model1.index1 if x[0] == s])
            return row_sum == 1
        self.model1.regime_limit = Constraint(self.model1.stands, rule=regime_rule)
        print("regime")

    def solve(self):
        # Specify the solver and solve the model
        opt = SolverFactory('cbc') #Here we use open source cbc solver - could be adjusted to another one here, for example glpk or cplex
        self.results = opt.solve(self.model1,tee=True) #We solve a problem, but do not show the solver output


#Create an optimization object        
t1 = optimization()
t2 = copy.deepcopy(t1)

try:
    t2.model1.del_component(t2.model1.NPV_INV)
except:
    print("NONE")

#A function that evaluates NPV (already discounted in Gaya simulated output)  
t2.model1.NPV= Var(within=NonNegativeReals) 
def NPV_INVENTORY(model1):
    row_sum = sum(t2.all_data.npv.loc[(s,r,6)]*t2.all_data.area.loc[(s,r,6)]* t2.model1.X1[(s,r)]  for (s,r) in t2.model1.index1)
    return t2.model1.NPV ==row_sum
t2.model1.NPV_INV= Constraint(rule=NPV_INVENTORY)  
print("NPV constraint built")


#Income slack objective evaluation
t2.model1.Inc_min= Var(within=NonNegativeReals) #variable to store minimum income over all periods 
t2.model1.Income = Var(t2.model1.year, within=NonNegativeReals) #variables for periodic incomes

def Income_periodic(model1, t):
    row_sum = sum(t2.all_data.INC.loc[(s,r,t)]*t2.all_data.area.loc[(s,r,t)]* t2.model1.X1[(s,r)]  for (s,r) in t2.model1.index1)
    return t2.model1.Income[t] == row_sum
t2.model1.Income_period = Constraint(t2.model1.year, rule=Income_periodic)

#Desirable periodic income - could be adjusted here
t2.model1.target_income = Param(default=400000, mutable=True)

#Create a slack variable for each period 
t2.model1.slack = Var(t2.model1.year, within=NonNegativeReals)

def income_target_rule(model1, t):
#Income[t] plus slack must meet or exceed target_income
    return t2.model1.Income[t] + t2.model1.slack[t] >= t2.model1.target_income
t2.model1.income_target = Constraint(t2.model1.year, rule=income_target_rule)

t2.model1.TotalSlack = Var(within=NonNegativeReals, initialize=0)
def total_slack_constraint(model1):
        return t2.model1.TotalSlack == sum(t2.model1.slack[k] for k in t2.model1.year)
t2.model1.TotalSlackConstraint = Constraint(rule=total_slack_constraint)
print("slack constraint built")


#CHSI objective evaluation
t2.all_data['CHSI'] = 1- ((1-t2.all_data.LSWP)*(1-t2.all_data.TTWP)*(1-t2.all_data.LTT)*(1-t2.all_data.CAP)*(1-t2.all_data.HAZ))


t2.model1.Landscape_HSI_Tot = Var(within=NonNegativeReals)

def total_HSI_constraint(model1):
    total_HSI = sum(
        model1.X1[(s, r)] * t2.all_data.CHSI.loc[(s, r, k)] * t2.all_data.area.loc[(s, r, k)]
        for (s, r) in model1.index1
        for k in model1.year
    )
    return model1.Landscape_HSI_Tot == total_HSI

t2.model1.LAND_HSI_TOTAL = Constraint(rule=total_HSI_constraint)

print("CHSI constraint built")

#standing end volume evalution

t2.model1.EndVolume = Var(within=NonNegativeReals)

def End_volume_constraint(model1):
    row_sum = sum(
        (t2.all_data.V_Pine_end.loc[(s, r, 6)] + 
         t2.all_data.V_Spruce_end.loc[(s, r, 6)] + 
         t2.all_data.V_Birch_end.loc[(s, r, 6)]) * 
        t2.all_data.area.loc[(s, r, 6)] * t2.model1.X1[(s, r)]
        for (s, r) in t2.model1.index1
    )
    return t2.model1.EndVolume == row_sum
t2.model1.EndInv = Constraint(rule=End_volume_constraint)

print("end v constraint built")



#minimum and maximum values and target parameters. If knowm - adjust here, if unknown yet - will be computed later

t2.model1.NPV_min = Param(default=1375007.80, mutable=True)
t2.model1.NPV_max = Param(default=2371796.30, mutable=True)

t2.model1.TotalSlack_min = Param(default=0, mutable=True)
t2.model1.TotalSlack_max = Param(default=2400000.00, mutable=True)

t2.model1.HSI_min = Param(default=4.85, mutable=True)
t2.model1.HSI_max = Param(default=171.43, mutable=True)

t2.model1.EndVol_min = Param(default=297.30, mutable=True)
t2.model1.EndVol_max = Param(default=10641.58, mutable=True)


t2.model1.NPV_target = Param(default=1500000, mutable=True)
t2.model1.EndVol_target = Param(default=8000, mutable=True)
t2.model1.TotalSlack_target = Param(default=1500000, mutable=True)
t2.model1.HSI_target = Param(default=150, mutable=True)


try:
    t2.model1.del_component(t2.model1.OBJ)
except:
    print("NONE")

#transforming problem into achievement-scalarizing function

t2.model1.D = Var(within=Reals)

def NPV_D_constraint(model1):
    row_sum = ((t2.model1.NPV - t2.model1.NPV_target)/(t2.model1.NPV_max-t2.model1.NPV_min))
    return t2.model1.D <= row_sum 
t2.model1.NPV_D = Constraint(rule=NPV_D_constraint)

def TotalSlack_D_constraint(model1):
    row_sum = ((t2.model1.TotalSlack_target-t2.model1.TotalSlack)/(t2.model1.TotalSlack_max - t2.model1.TotalSlack_min))
    return t2.model1.D <= row_sum 
t2.model1.TotalSlack_D = Constraint(rule=TotalSlack_D_constraint)

def HSI_D_constraint(model1):
    row_sum = ((t2.model1.Landscape_HSI_Tot - t2.model1.HSI_target)/(t2.model1.HSI_max - t2.model1.HSI_min))
    return t2.model1.D <= row_sum 
t2.model1.HSI_D = Constraint(rule=HSI_D_constraint)

def End_V_D_constraint(model1):
    row_sum = ((t2.model1.EndVolume - t2.model1.EndVol_target)/(t2.model1.EndVol_max - t2.model1.EndVol_min))
    return t2.model1.D <= row_sum 
t2.model1.END_V_D = Constraint(rule=End_V_D_constraint)
print("D built")

#Objective function
def outcome_rule(model1):
    return ((t2.model1.D + 0.001*(((t2.model1.NPV)/(t2.model1.NPV_max-t2.model1.NPV_min)) + 
            ((t2.model1.EndVolume)/(t2.model1.EndVol_max - t2.model1.EndVol_min)) - 
            ((t2.model1.TotalSlack)/(t2.model1.TotalSlack_max - t2.model1.TotalSlack_min)) +
            ((t2.model1.Landscape_HSI_Tot)/(t2.model1.HSI_max - t2.model1.HSI_min)) 
            ))*10000) 
t2.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)


# Solve the modified optimization model
t2.solve()


#Function to extract decision variables from the optimized model
def GET_DECISION_DATA():
        st = []
        reg = []
        vals = []
        for (s,r) in t2.model1.index1:
            st = st+[s]
            reg = reg+[r] 
            vals = vals+[t2.model1.X1[(s,r)].value]
        data = {"stand":st,"schedule":reg,"value":vals}
        df= pd.DataFrame(data)
        df = df.set_index(['stand','schedule'])
        return df

# Extract decision data and merge with original dataset    
dec  = GET_DECISION_DATA()
merged_df = dec.merge(t2.all_data, left_index=True, right_index=True, how='left')

print("Objective value")
print(value(t2.model1.OBJ))
print("NPV value")
print(t2.model1.NPV.value)
print("Clack:")
print(t2.model1.TotalSlack.value)
print("EndVolume:")
print(t2.model1.EndVolume.value)
print("Landscape_HSI_Tot:")
print(t2.model1.Landscape_HSI_Tot.value)
print("Periodic Income and Slack values:")
for period in t2.model1.year:
    income_val = value(t2.model1.Income[period])
    slack_val = value(t2.model1.slack[period])
    print(f"Period {period}: Income = {income_val}, Slack = {slack_val}")

#save the output in the same format as the input file + the decision column
merged_df.to_parquet(path+"cells_output_target18.parquet")

data loaded
regime
NONE
NPV
slack
CHSI
end v
D

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'D:\NMBU\TEMP\tmpwczkylhj.cplex.log' open.
CPLEX> Problem 'D:\NMBU\TEMP\tmpvrzzn1cx.pyomo.lp' read.
Read time = 0.52 sec. (21.05 ticks)
CPLEX> Problem name         : D:\NMBU\TEMP\tmpvrzzn1cx.pyomo.lp
Objective sense      : Maximize
Variables            :  278473  [Nneg: 16,  Free: 1,  Binary: 278456]
Objective nonzeros   :       5
Linear constraints   :    1739  [Less: 4,  Greater: 6,  Equal: 1729]
  Nonzeros           : 1139194
  RHS nonzeros       :    1729

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective nonzero

In [5]:
#This code creates a payoff table - can be used after the previous code is executed
#It runs the model by optimizing one objective one by one


def RE_TRY(t2,VAL):
    t2.model1.del_component(t2.model1.OBJ)
    def outcome_rule1(model1):
        if VAL == "NPV":
            return t2.model1.NPV  
        elif VAL == "END":
            return t2.model1.EndVolume 
        elif VAL == "Total_Slack":
            return - t2.model1.TotalSlack
        elif VAL == "HSI":
            return t2.model1.Landscape_HSI_Tot
    t2.model1.OBJ = Objective(rule=outcome_rule1, sense=maximize)
    # Solve the modified optimization model
    t2.solve()
    t2a ={"NPV":t2.model1.NPV.value,"EndVolume": t2.model1.EndVolume.value,"Total_Slack": t2.model1.TotalSlack.value,"Landscape_HSI_Tot": t2.model1.Landscape_HSI_Tot.value}
    return t2a
t2a = RE_TRY(t2,"NPV")
t2b = RE_TRY(t2,"END")
t2c = RE_TRY(t2,"Total_Slack")
t2d = RE_TRY(t2,"HSI")

 

data = [t2a,t2b,t2c,t2d]
 
# Creating a DataFrame
df = pd.DataFrame(data)
(df.T).to_csv(path+"payoff_cells.csv")  #Define the path to save the payoff table

print(df.T)

                              0             1             2             3
NPV                2.371796e+06  1.386045e+06  1.539750e+06  1.375008e+06
EndVolume          2.973004e+02  1.064158e+04  2.289559e+03  1.053287e+04
Total_Slack        1.827234e+06  2.370906e+06  0.000000e+00  2.400000e+06
Landscape_HSI_Tot  4.851869e+00  1.658241e+02  7.438049e+01  1.714327e+02


In [ ]:
#This code will extract the minimum and maximum values from the payoff table

import csv

def get_max_and_min_values_in_row(csv_file, row_number):
    with open(csv_file, newline='') as file:
        reader = csv.reader(file)
        # Skip the header row
        next(reader)
        for _ in range(row_number - 1):
            try:
                row = next(reader)
            except StopIteration:
                return None, None, None, None
        
        variable_name = row[0].strip()  
        values = []
        for value in row[1:]:
            try:
                numeric_value = float(value)
                values.append(numeric_value)
            except ValueError:
                pass  
        if values:
            max_value = max(values)
            min_value = min(values)
        else:
            max_value = None
            min_value = None
        return variable_name + "_Max", max_value, variable_name + "_Min", min_value

#The path to payoff table
csv_file = path+"Outputs/Payoff_tables/payoff_direct.csv"

for row_number in range(2, 7):  # Identify the rown with values, skipping the header
    max_var_name, max_value, min_var_name, min_value = get_max_and_min_values_in_row(csv_file, row_number)
    if max_var_name is not None:
        locals()[max_var_name] = max_value
    if min_var_name is not None:
        locals()[min_var_name] = min_value


#Store the extracted minimum and maximum values as parameters for the optimization model
t2.model1.NPV_min = NPV_Min
t2.model1.NPV_max = NPV_Max

t2.model1.HSI_min = Landscape_HSI_Tot_Min
t2.model1.HSI_max = Landscape_HSI_Tot_Max

t2.model1.TotalSlack_min = TotalSlack_Min
t2.model1.TotalSlack_max = TotalSlack_Max

t2.model1.EndVol_min = EndVolume_Min
t2.model1.EndVol_max = EndVolume_Max

In [5]:
#The main integer problem for running on stands or cells with no clustering

class optimization:
    def __init__(self):
        
        data_opt =  pd.read_csv(path + "2025-03-27_stands_30_same_complete.csv") 
        print("data loaded")
        all_data = data_opt
        all_data = all_data.set_index(['stand','schedule','period']) #reassigns the Df with new multi-level index
        
        all_data = all_data.fillna(0) #fills any missing values with 0
        all_data['period'] = all_data.index.get_level_values(2)
        
        self.data_opt=all_data #store the fetched or preprocessed data for use within the class
        self.combinations = 1
        self.all_data = self.data_opt
        self.Index_values = self.all_data.drop(['period'], axis=1).reset_index().set_index(['stand','schedule']).index.unique() #we drop year, because we dont make decision for each year
        self.area = self.all_data.loc[slice(None),1,1]['area']
        self.all_data = self.all_data.fillna(0)

        self.createModel()

    def createModel(self):
        # Declare sets - These used to recongnize the number of stands, regimes and number of periods in the analysis.
        # Define sets, variables, and constraints for the optimization problem
        
        self.model1 = ConcreteModel() #defining the model
        
        self.model1.stands = Set(initialize = list(set(self.all_data.index.get_level_values(0)))) #initialized with unique values from the first level of the index created earlier
        self.model1.year = Set(initialize = list(set(self.all_data.index.get_level_values(2)))) #initialized with unique values from the third level of the index created earlier
               
        self.model1.regimes = Set(initialize = list(set(self.all_data.index.get_level_values(1))))
        self.model1.scen_index = Set(initialize= [i for i in range(0,self.combinations)])
        self.model1.Index_values = self.Index_values

        # Indexes (stand, regime)-- excludes those combinations that have no regimes simulated -- because some regimes are not simulated for some stands

        def index_rule(model1):
            index = []
            for (s,r) in model1.Index_values: 
                index.append((s,r))
            return index
        self.model1.index1 = Set(dimen=2, initialize=index_rule)

        #Decision variable
        self.model1.X1 = Var(self.model1.index1, within=Boolean, initialize=1) 

        self.all_data['period'] = self.all_data.index.get_level_values(2)

        #Objective and constraint don't need to be adjusted here, leave as it is, used to create the optimization model in the code. 
        #objective function:
        def outcome_rule(model1):
            return sum((self.all_data.Harvested_V_S.loc[(s,r,k)]*self.all_data.area.loc[(s,r,k)]* self.model1.X1[(s,r)])/((1+INTEREST)**(2.5+self.all_data.period[(s,r,k)]))  for (s,r) in self.model1.index1 for k in self.model1.year)
        self.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)

        #Constraint:
        def regime_rule(model1, s):
            row_sum = sum(model1.X1[(s,r)] for r in [x[1] for x in model1.index1 if x[0] == s])
            return row_sum == 1
        self.model1.regime_limit = Constraint(self.model1.stands, rule=regime_rule)
        print("regime")

    def solve(self):
        # Specify the solver and solve the model
        opt = SolverFactory('cbc') #select solver
        self.results = opt.solve(self.model1,tee=True) #We solve a problem, but do not show the solver output


# Create an optimization object        
t1 = optimization()
t2 = copy.deepcopy(t1)

# Modify the optimization model to focus on. 

try:
    t2.model1.del_component(t2.model1.NPV_INV)
except:
    print("NONE")

#NPV (already discounted in Gaya simulated output)  
t2.model1.NPV= Var(within=NonNegativeReals) 
def NPV_INVENTORY(model1):
    row_sum = sum(t2.all_data.npv.loc[(s,r,6)]*t2.all_data.area.loc[(s,r,6)]* t2.model1.X1[(s,r)]  for (s,r) in t2.model1.index1)
    return t2.model1.NPV ==row_sum
t2.model1.NPV_INV= Constraint(rule=NPV_INVENTORY)  
print("NPV constraint built")


#Income slack
t2.model1.Inc_min= Var(within=NonNegativeReals) #variable to store minimum income over all periods 
t2.model1.Income = Var(t2.model1.year, within=NonNegativeReals) #variables for periodic incomes

def Income_periodic(model1, t):
    row_sum = sum(t2.all_data.INC.loc[(s,r,t)]*t2.all_data.area.loc[(s,r,t)]* t2.model1.X1[(s,r)]  for (s,r) in t2.model1.index1)
    return t2.model1.Income[t] == row_sum
t2.model1.Income_period = Constraint(t2.model1.year, rule=Income_periodic)

t2.model1.target_income = Param(default=400000, mutable=True) #adjust target if needed
# Create a slack variable for each period (nonnegative)
t2.model1.slack = Var(t2.model1.year, within=NonNegativeReals)

def income_target_rule(model1, t):
# Income[t] plus slack must meet or exceed target_income.
    return t2.model1.Income[t] + t2.model1.slack[t] >= t2.model1.target_income
t2.model1.income_target = Constraint(t2.model1.year, rule=income_target_rule)

t2.model1.TotalSlack = Var(within=NonNegativeReals, initialize=0)
def total_slack_constraint(model1):
        return t2.model1.TotalSlack == sum(t2.model1.slack[k] for k in t2.model1.year)
t2.model1.TotalSlackConstraint = Constraint(rule=total_slack_constraint)
print("income slack constraint built")

#CHSI
t2.all_data['CHSI'] = 1- ((1-t2.all_data.LSWP)*(1-t2.all_data.TTWP)*(1-t2.all_data.LTT)*(1-t2.all_data.CAP)*(1-t2.all_data.HAZ))

t2.model1.Landscape_HSI_Tot = Var(within=NonNegativeReals)


def total_HSI_constraint(model1):
    total_HSI = sum(
        model1.X1[(s, r)] * t2.all_data.CHSI.loc[(s, r, k)] * t2.all_data.area.loc[(s, r, k)]
        for (s, r) in model1.index1
        for k in model1.year
    )
    return model1.Landscape_HSI_Tot == total_HSI

t2.model1.LAND_HSI_TOTAL = Constraint(rule=total_HSI_constraint)

print("CHSI constraint built")

#standing end volume 

t2.model1.EndVolume = Var(within=NonNegativeReals)

def End_volume_constraint(model1):
    row_sum = sum(
        (t2.all_data.V_Pine_end.loc[(s, r, 6)] + 
         t2.all_data.V_Spruce_end.loc[(s, r, 6)] + 
         t2.all_data.V_Birch_end.loc[(s, r, 6)]) * 
        t2.all_data.area.loc[(s, r, 6)] * t2.model1.X1[(s, r)]
        for (s, r) in t2.model1.index1
    )
    return t2.model1.EndVolume == row_sum
t2.model1.EndInv = Constraint(rule=End_volume_constraint)

print("end v constraint built")



#minimum and maximum values for each objective. Adjust here based on the payoff table

t2.model1.NPV_min = Param(default=1375183.70, mutable=True)
t2.model1.NPV_max = Param(default=2312271.00, mutable=True)


t2.model1.TotalSlack_min = Param(default=0, mutable=True)
t2.model1.TotalSlack_max = Param(default=2400000.00, mutable=True)


t2.model1.HSI_min = Param(default=5.05, mutable=True)
t2.model1.HSI_max = Param(default=174.45, mutable=True)


t2.model1.EndVol_min = Param(default=276.81, mutable=True)
t2.model1.EndVol_max = Param(default=10433.67, mutable=True)


#targets - adjust here

t2.model1.NPV_target = Param(default=2000000, mutable=True)
t2.model1.EndVol_target = Param(default=8000, mutable=True)
t2.model1.TotalSlack_target = Param(default=100000, mutable=True)
t2.model1.HSI_target = Param(default=150, mutable=True)



try:
    t2.model1.del_component(t2.model1.OBJ)
except:
    print("NONE")

#transforming to achievement-scalarizing function
t2.model1.D = Var(within=Reals)

def NPV_D_constraint(model1):
    row_sum = ((t2.model1.NPV - t2.model1.NPV_target)/(t2.model1.NPV_max-t2.model1.NPV_min))
    return t2.model1.D <= row_sum 
t2.model1.NPV_D = Constraint(rule=NPV_D_constraint)

def TotalSlack_D_constraint(model1):
    row_sum = ((t2.model1.TotalSlack_target-t2.model1.TotalSlack)/(t2.model1.TotalSlack_max - t2.model1.TotalSlack_min))
    return t2.model1.D <= row_sum 
t2.model1.TotalSlack_D = Constraint(rule=TotalSlack_D_constraint)

def HSI_D_constraint(model1):
    row_sum = ((t2.model1.Landscape_HSI_Tot - t2.model1.HSI_target)/(t2.model1.HSI_max - t2.model1.HSI_min))
    return t2.model1.D <= row_sum 
t2.model1.HSI_D = Constraint(rule=HSI_D_constraint)

def End_V_D_constraint(model1):
    row_sum = ((t2.model1.EndVolume - t2.model1.EndVol_target)/(t2.model1.EndVol_max - t2.model1.EndVol_min))
    return t2.model1.D <= row_sum 
t2.model1.END_V_D = Constraint(rule=End_V_D_constraint)
print("D")

def outcome_rule(model1):
    return ((t2.model1.D + 0.001*(((t2.model1.NPV)/(t2.model1.NPV_max-t2.model1.NPV_min)) + 
            ((t2.model1.EndVolume)/(t2.model1.EndVol_max - t2.model1.EndVol_min)) - 
            ((t2.model1.TotalSlack)/(t2.model1.TotalSlack_max - t2.model1.TotalSlack_min)) +
            ((t2.model1.Landscape_HSI_Tot)/(t2.model1.HSI_max - t2.model1.HSI_min)) 
            ))*10000) 
t2.model1.OBJ = Objective(rule=outcome_rule, sense=maximize)

# Solve the modified optimization model
t2.solve()


#Function to extract decision variables from the optimized model
def GET_DECISION_DATA():
        st = []
        reg = []
        vals = []
        for (s,r) in t2.model1.index1:
            st = st+[s]
            reg = reg+[r] 
            vals = vals+[t2.model1.X1[(s,r)].value]
        data = {"stand":st,"schedule":reg,"value":vals}
        df= pd.DataFrame(data)
        df = df.set_index(['stand','schedule'])
        return df

# Extract decision data and merge with original dataset    
dec  = GET_DECISION_DATA()
merged_df = dec.merge(t2.all_data, left_index=True, right_index=True, how='left')

print("Objective value")
print(value(t2.model1.OBJ))
print("NPV value")
print(t2.model1.NPV.value)
print("Clack:")
print(t2.model1.TotalSlack.value)
print("EndVolume:")
print(t2.model1.EndVolume.value)
print("Landscape_HSI_Tot:")
print(t2.model1.Landscape_HSI_Tot.value)
print("Periodic Income and Slack values:")
for period in t2.model1.year:
    income_val = value(t2.model1.Income[period])
    slack_val = value(t2.model1.slack[period])
    print(f"Period {period}: Income = {income_val}, Slack = {slack_val}")



merged_df.to_csv(path+"stands_output_target16.csv")

data loaded
regime
NONE
NPV
slack
CHSI
end v
D

Welcome to IBM(R) ILOG(R) CPLEX(R) Interactive Optimizer 22.1.0.0
  with Simplex, Mixed Integer & Barrier Optimizers
5725-A06 5725-A29 5724-Y48 5724-Y49 5724-Y54 5724-Y55 5655-Y21
Copyright IBM Corp. 1988, 2022.  All Rights Reserved.

Type 'help' for a list of available commands.
Type 'help' followed by a command name for more
information on commands.

CPLEX> Logfile 'cplex.log' closed.
Logfile 'D:\NMBU\TEMP\tmpkea4xf3u.cplex.log' open.
CPLEX> Problem 'D:\NMBU\TEMP\tmpuncw171u.pyomo.lp' read.
Read time = 0.02 sec. (0.40 ticks)
CPLEX> Problem name         : D:\NMBU\TEMP\tmpuncw171u.pyomo.lp
Objective sense      : Maximize
Variables            :    5273  [Nneg: 16,  Free: 1,  Binary: 5256]
Objective nonzeros   :       5
Linear constraints   :      54  [Less: 4,  Greater: 6,  Equal: 44]
  Nonzeros           :   21310
  RHS nonzeros       :      44

Variables            : Min LB: 0.000000         Max UB: 1.000000       
Objective nonzeros   :